<a href="https://colab.research.google.com/github/microprediction/timeseries-notebooks/blob/main/just_for_fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classifying


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit

# Define different model functions
def linear_model(x, a, b):
    return a * x + b

def quadratic_model(x, a, b, c):
    return a * x**2 + b * x + c

def logarithmic_model(x, a, b):
    return a * np.log(x) + b

def exponential_model(x, a, b):
    return a * np.exp(b * x)

def logistic_model(x, a, b, c):
    return c / (1 + np.exp(-(x - b) / a))

# Function to classify the time-series data
def classify_time_series(x, y):
    results = {}

    # Linear fit
    linear_reg = LinearRegression()
    linear_reg.fit(x.reshape(-1, 1), y)
    y_pred = linear_reg.predict(x.reshape(-1, 1))
    results['Linear'] = r2_score(y, y_pred)

    # Quadratic fit
    poly = PolynomialFeatures(degree=2)
    x_poly = poly.fit_transform(x.reshape(-1, 1))
    poly_reg = LinearRegression()
    poly_reg.fit(x_poly, y)
    y_pred = poly_reg.predict(x_poly)
    results['Quadratic'] = r2_score(y, y_pred)

    # Logarithmic fit (avoid zero or negative x)
    if np.all(x > 0):
        try:
            popt, _ = curve_fit(logarithmic_model, x, y)
            y_pred = logarithmic_model(x, *popt)
            results['Logarithmic'] = r2_score(y, y_pred)
        except:
            results['Logarithmic'] = -np.inf

    # Exponential fit
    try:
        popt, _ = curve_fit(exponential_model, x, y)
        y_pred = exponential_model(x, *popt)
        results['Exponential'] = r2_score(y, y_pred)
    except:
        results['Exponential'] = -np.inf

    # Logistic fit
    try:
        popt, _ = curve_fit(logistic_model, x, y, maxfev=10000)
        y_pred = logistic_model(x, *popt)
        results['Logistic'] = r2_score(y, y_pred)
    except:
        results['Logistic'] = -np.inf

    # Classify based on the best R2 score
    best_fit = max(results, key=results.get)
    return best_fit, results

# Example usage
x = np.arange(1, 101)
# Generate some example data for testing
y = 3 * x + 10  # Linear example

# Classify the time series
best_fit, scores = classify_time_series(x, y)
print(f'Best fit model: {best_fit}')
print(f'R2 scores: {scores}')


Best fit model: Linear
R2 scores: {'Linear': 1.0, 'Quadratic': 1.0, 'Logarithmic': 0.8025595986962138, 'Exponential': -1.7395862748559822e+53, 'Logistic': 0.9936772496689723}
